In [68]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_absolute_error, mean_squared_error
from IPython.display import display

# Memuat file Excel
file_path = 'data_full.xlsx'  # Ganti dengan path file Anda
df_full_data = pd.read_excel(file_path)

# Menampilkan data yang telah dimuat
display(df_full_data.head())


,Wilayah,Kategori,1,2,3,4,5,6,7,8,...,43,44,45,46,47,48,49,50,51,52
0,Banda Sakti,Bisnis,1828,1556,1506,1557,1842,1761,1865,1759,...,1578,1706,2341,1556,1506,2019,2102,1943,1464,1786
1,Banda Sakti,Industri,1129,1418,1459,1470,1269,1065,1135,1194,...,1406,1079,1273,1418,1459,1382,1380,1358,1314,1153
2,Banda Sakti,Lain-lain,1008,618,778,897,567,773,760,1117,...,937,462,1125,618,778,660,559,1411,811,1103
3,Banda Sakti,Rumah Tangga,491,329,1151,771,794,835,802,850,...,854,710,958,329,1151,868,1106,823,865,623
4,Banda Sakti,Sosial,980,1060,884,933,794,1046,1127,1108,...,1258,1164,985,1060,884,903,1167,1144,1022,1200


In [69]:
# Mengubah data dari format wide ke long
df_long = pd.melt(df_full_data, id_vars=['Wilayah', 'Kategori'], var_name='Minggu', value_name='Pemakaian')

# Mengubah kolom 'Minggu' menjadi numerik
df_long['Minggu'] = df_long['Minggu'].astype(int)

# Menampilkan data dalam format long
display(df_long.head())


,Wilayah,Kategori,Minggu,Pemakaian
0,Banda Sakti,Bisnis,1,1828
1,Banda Sakti,Industri,1,1129
2,Banda Sakti,Lain-lain,1,1008
3,Banda Sakti,Rumah Tangga,1,491
4,Banda Sakti,Sosial,1,980


In [70]:
# Mendapatkan daftar wilayah
wilayah_list = df_long['Wilayah'].unique()

# Mendapatkan daftar kategori
kategori_list = df_long['Kategori'].unique()

# Menampilkan daftar wilayah dan kategori
print("Daftar Wilayah:", wilayah_list)
print("Daftar Kategori:", kategori_list)


Daftar Wilayah: ['Banda Sakti' 'Blang Mangat' 'Muara Dua' 'Muara Satu']
Daftar Kategori: ['Bisnis' 'Industri' 'Lain-lain' 'Rumah Tangga' 'Sosial']


In [71]:
# Metrik kesalahan per wilayah dan kategori
error_metrics = {}

# Menyimpan hasil per wilayah dan kategori
df_results = {}

# Model yang akan digunakan
models = ['MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA']


In [77]:
# Menyimpan hasil dan metrik kesalahan
df_results = {}
error_metrics = {}

# Inisialisasi DataFrame untuk menyimpan semua prediksi
prediksi_final = pd.DataFrame()

for wilayah in wilayah_list:
    # Memfilter data untuk wilayah tertentu
    df_wilayah = df_long[df_long['Wilayah'] == wilayah].copy()
    
    # Mengecek apakah data tersedia
    if df_wilayah.empty:
        continue  # Jika tidak ada data, lewati iterasi ini
    
    # Mengurutkan data berdasarkan 'Minggu' dan 'Kategori'
    df_wilayah.sort_values(['Minggu', 'Kategori'], inplace=True)
    df_wilayah.reset_index(drop=True, inplace=True)
    
    # Pivot data sehingga setiap kategori menjadi kolom
    df_pivot = df_wilayah.pivot(index='Minggu', columns='Kategori', values='Pemakaian').reset_index()
    
    # Mengisi nilai NaN jika ada
    df_pivot.fillna(0, inplace=True)
    
    # Menentukan Kategori yang Tersedia dalam df_pivot
    available_categories = [kategori for kategori in kategori_list if kategori in df_pivot.columns]
    
    # Jika tidak ada kategori yang tersedia, lewati wilayah ini
    if not available_categories:
        print(f"Tidak ada kategori yang tersedia untuk wilayah {wilayah}.")
        continue
    
    # Menyimpan 'Pemakaian Asli' per kategori yang tersedia
    pemakaian_asli = df_pivot[available_categories].values  # Bentuknya (jumlah_minggu, jumlah_kategori)
    
    # Inisialisasi DataFrame hasil untuk wilayah ini
    df_region_results = df_pivot[['Minggu']].copy()
    
    for idx, kategori in enumerate(available_categories):
        series = pemakaian_asli[:, idx]
        
        if np.all(series == 0):
            continue
        
        df_region_results[f'{kategori}'] = series
        
        # Menghitung Moving Average
        ma_values = moving_average(series, period=3)
        df_region_results[f'{kategori} - MA (3-Mingguan)'] = ma_values
        
        # Model AR
        ar_values = np.append([np.nan], series[:-1] * phi_ar)
        df_region_results[f'{kategori} - AR(1)'] = ar_values
        
        # Model AR+MA
        residuals = series - ma_values
        residuals[np.isnan(residuals)] = 0
        ar_ma_values = ar_values + theta_ma * np.append([0], residuals[:-1])
        df_region_results[f'{kategori} - AR+MA'] = ar_ma_values
        
        # Model VARMA
        varma_values = calculate_varma_values(series, residuals, phi_varma, theta_varma, beta1, beta2)
        df_region_results[f'{kategori} - VARMA'] = varma_values
        
        # Menghitung MAPE, RMSE, dan MAE
        error_metrics_model = {}
        for model in models:
            model_col = f'{kategori} - {model}'
            actual = df_region_results[f'{kategori}'][~np.isnan(df_region_results[model_col])]
            predicted = df_region_results[model_col][~np.isnan(df_region_results[model_col])]
            if len(actual) > 0:  # Pastikan ada data untuk menghitung metrik
                mae = mean_absolute_error(actual, predicted)
                rmse = mean_squared_error(actual, predicted, squared=False)
                mape = np.mean(np.abs((actual - predicted) / actual)) * 100
                error_metrics_model[model] = {'MAE': round(mae, 2), 'RMSE': round(rmse, 2), 'MAPE': round(mape, 2)}
        
        # Menyimpan metrik kesalahan per kategori
        error_metrics[(wilayah, kategori)] = error_metrics_model
    
    df_results[wilayah] = df_region_results.copy()

    # **Prediksi untuk Minggu 53 hingga 62**
    for week in range(53, 63):
        new_data = {'Minggu': week}
        
        for kategori in available_categories:
            series = df_region_results[kategori].values
            last_value = series[-1]
            
            # Contoh prediksi yang lebih realistis
            ar_ma_pred = max(0, last_value * (1 + np.random.uniform(-0.1, 0.1)))  # Menggunakan noise kecil
            
            new_data[kategori] = ar_ma_pred
        
        df_region_results = pd.concat([df_region_results, pd.DataFrame([new_data])], ignore_index=True)

    # Menyiapkan DataFrame untuk hasil prediksi
    prediksi_df = df_region_results[df_region_results['Minggu'].isin(range(53, 63))].copy()
    prediksi_df['Region'] = wilayah  # Menambahkan kolom Region
    
    # Menggabungkan hasil prediksi ke dalam prediksi_final DataFrame
    prediksi_final = pd.concat([prediksi_final, prediksi_df], ignore_index=True)

# Menampilkan tabel akhir hasil prediksi
print("Hasil Prediksi untuk Setiap Wilayah:")
prediksi_final_display = prediksi_final[['Region', 'Minggu'] + available_categories]
display(prediksi_final_display)

# Menampilkan metrik kesalahan per wilayah dalam tabel terpisah
print("Metrik Kesalahan untuk Setiap Wilayah:")
for wilayah in wilayah_list:
    error_df_region = pd.DataFrame.from_dict(
        {(kategori, model): metrics for (w, kategori), models_metrics in error_metrics.items() if w == wilayah for model, metrics in models_metrics.items()}, 
        orient='index'
    )
    error_df_region.reset_index(inplace=True)
    error_df_region.columns = ['Kategori', 'Model', 'MAE', 'RMSE', 'MAPE']
    
    print(f"\nMetrik Kesalahan untuk Wilayah {wilayah}:")
    display(error_df_region)


Hasil Prediksi untuk Setiap Wilayah:


,Region,Minggu,Industri,Lain-lain,Rumah Tangga,Sosial
0,Banda Sakti,53,1261.657571,1096.077623,616.470591,1221.554315
1,Banda Sakti,54,1250.797833,1181.446796,566.746758,1341.971411
2,Banda Sakti,55,1332.810556,1221.690262,614.288674,1469.087374
3,Banda Sakti,56,1349.310715,1306.112225,602.333034,1465.286502
4,Banda Sakti,57,1258.131230,1435.013070,651.448157,1561.402153
5,Banda Sakti,58,1297.054583,1355.583986,614.421186,1500.468682
6,Banda Sakti,59,1414.883423,1382.026385,662.956495,1599.781115
7,Banda Sakti,60,1487.062186,1332.523339,675.066486,1539.579874
8,Banda Sakti,61,1618.420053,1400.365936,714.817857,1566.424620
9,Banda Sakti,62,1733.996681,1313.134173,732.460336,1664.638254


Metrik Kesalahan untuk Setiap Wilayah:

Metrik Kesalahan untuk Wilayah Banda Sakti:


,Kategori,Model,MAE,RMSE,MAPE
0,Bisnis,MA (3-Mingguan),219.53,285.11,11.64
1,Bisnis,AR(1),442.82,566.79,22.07
2,Bisnis,AR+MA,459.49,589.05,23.14
3,Bisnis,VARMA,1583.84,2056.95,86.67
4,Industri,MA (3-Mingguan),90.37,107.75,7.21
5,Industri,AR(1),262.47,303.65,19.61
6,Industri,AR+MA,267.46,308.37,20.09
7,Industri,VARMA,958.91,1101.86,75.98
8,Lain-lain,MA (3-Mingguan),140.45,179.24,18.96
9,Lain-lain,AR(1),276.96,339.16,33.34



Metrik Kesalahan untuk Wilayah Blang Mangat:


,Kategori,Model,MAE,RMSE,MAPE
0,Bisnis,MA (3-Mingguan),289.22,369.55,11.31
1,Bisnis,AR(1),663.43,781.37,23.88
2,Bisnis,AR+MA,684.12,810.94,24.81
3,Bisnis,VARMA,2293.49,2806.87,90.98
4,Industri,MA (3-Mingguan),122.84,156.26,4.93
5,Industri,AR(1),493.16,565.43,19.24
6,Industri,AR+MA,494.57,575.39,19.28
7,Industri,VARMA,1814.07,2022.01,74.48
8,Lain-lain,MA (3-Mingguan),118.19,146.51,14.56
9,Lain-lain,AR(1),220.87,280.69,24.49



Metrik Kesalahan untuk Wilayah Muara Dua:


,Kategori,Model,MAE,RMSE,MAPE
0,Bisnis,MA (3-Mingguan),217.17,264.07,12.17
1,Bisnis,AR(1),450.17,558.25,22.45
2,Bisnis,AR+MA,462.23,576.41,23.19
3,Bisnis,VARMA,1651.61,1958.00,91.78
4,Lain-lain,MA (3-Mingguan),111.23,143.24,27.79
5,Lain-lain,AR(1),198.45,247.51,41.55
6,Lain-lain,AR+MA,208.54,262.24,43.84
7,Lain-lain,VARMA,730.17,897.87,161.60
8,Rumah Tangga,MA (3-Mingguan),76.49,92.16,44.89
9,Rumah Tangga,AR(1),134.60,163.30,69.44



Metrik Kesalahan untuk Wilayah Muara Satu:


,Kategori,Model,MAE,RMSE,MAPE
0,Industri,MA (3-Mingguan),135.01,163.78,3.64
1,Industri,AR(1),747.34,791.22,19.90
2,Industri,AR+MA,746.45,796.22,19.88
3,Industri,VARMA,2713.41,2856.84,73.59
4,Lain-lain,MA (3-Mingguan),151.11,188.00,14.88
5,Lain-lain,AR(1),312.99,390.10,27.19
6,Lain-lain,AR+MA,322.88,408.02,28.13
7,Lain-lain,VARMA,1146.79,1358.45,110.46
8,Rumah Tangga,MA (3-Mingguan),159.99,210.36,26.50
9,Rumah Tangga,AR(1),284.56,343.59,40.31


In [83]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Membuat subplot untuk setiap wilayah dengan format 2x2
cols = 2
rows = (len(wilayah_list) + cols - 1) // cols  # Menghitung jumlah baris yang diperlukan

fig = make_subplots(
    rows=rows, 
    cols=cols,
    subplot_titles=wilayah_list
)

# Warna yang akan digunakan untuk setiap kategori
colors = {
    'Industri': 'blue',
    'Lain-lain': 'orange',
    'Rumah Tangga': 'green',
    'Sosial': 'red'
}

# Menambahkan data prediksi ke subplot
for idx, wilayah in enumerate(wilayah_list):
    prediksi_region = prediksi_final[prediksi_final['Region'] == wilayah]
    
    # Menentukan posisi subplot
    row = idx // cols + 1
    col = idx % cols + 1
    
    for kategori in available_categories:
        if kategori in prediksi_region.columns:
            # Menentukan apakah jejak ini adalah yang pertama untuk kategori ini
            show_legend = True if (idx == 0) else False  # Hanya menampilkan legend untuk kategori pertama
            
            # Menambahkan jejak hanya jika kategori memiliki nilai
            fig.add_trace(go.Scatter(
                x=prediksi_region['Minggu'],
                y=prediksi_region[kategori],
                mode='lines+markers',
                name=kategori,  # Menyederhanakan nama kategori
                line=dict(color=colors[kategori]),  # Menggunakan warna sesuai kategori
                marker=dict(size=8),
                showlegend=show_legend  # Menentukan apakah legend ditampilkan
            ), row=row, col=col)

# Menyusun layout grafik
fig.update_layout(
    title='Hasil Prediksi Pemakaian Listrik (Minggu 53-62) per Wilayah',
    xaxis_title='Minggu',
    yaxis_title='Pemakaian',
    legend_title='Kategori',
    template='plotly_white',
    height=400 * rows  # Menyesuaikan tinggi berdasarkan jumlah wilayah
)

# Menampilkan garis grid
fig.update_xaxes(showgrid=True, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridcolor='LightGrey')

# Menampilkan grafik
fig.show()


In [ ]:
import matplotlib.pyplot as plt

# Membuat subplot untuk setiap wilayah, dengan kategori sebagai subplots
for wilayah in wilayah_list:
    # Dapatkan daftar kategori yang tersedia untuk wilayah ini
    kategori_wilayah = [kategori for kategori in kategori_list if (wilayah, kategori) in df_results]
    num_kategori = len(kategori_wilayah)
    
    if num_kategori == 0:
        print(f"Tidak ada kategori untuk wilayah {wilayah}.")  # Jika tidak ada kategori, cetak pesan
        continue  # Jika tidak ada kategori untuk wilayah ini, lewati
    
    # Tentukan jumlah baris dan kolom untuk subplots
    cols = 2  # Misalnya, kita gunakan 2 kolom
    rows = (num_kategori + cols - 1) // cols  # Menghitung jumlah baris yang diperlukan
    
    # Membuat figure dan subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()  # Meratakan array axes untuk mempermudah akses
    
    # Loop melalui kategori untuk wilayah ini
    for idx, kategori in enumerate(kategori_wilayah):
        key = (wilayah, kategori)
        df_region = df_results[key]
        
        # Menambahkan Pemakaian Asli
        axes[idx].plot(df_region['Minggu'], df_region['Pemakaian'], marker='o', color='blue', label='Pemakaian Asli')
        
        # Menambahkan model-model
        colors = {'MA (3-Mingguan)': 'orange', 'AR(1)': 'green', 'AR+MA': 'red', 'VARMA': 'purple'}
        for model in models:
            if model in df_region.columns:  # Memeriksa apakah model ada dalam DataFrame
                axes[idx].plot(df_region['Minggu'], df_region[model], marker='o', label=model, color=colors[model])
        
        # Menambahkan judul dan label sumbu pada subplot
        axes[idx].set_title(f'Perbandingan {kategori} di {wilayah}')
        axes[idx].set_xlabel('Minggu')
        axes[idx].set_ylabel('Pemakaian Listrik')
        axes[idx].legend()
    
    # Menghapus subplot kosong jika ada
    for j in range(num_kategori, len(axes)):
        fig.delaxes(axes[j])
    
    # Menyesuaikan layout keseluruhan
    plt.tight_layout()
    plt.suptitle(f'Perbandingan Model untuk Wilayah {wilayah}', fontsize=16)
    plt.subplots_adjust(top=0.9)  # Menyesuaikan jarak judul utama
    plt.show()


In [ ]:
# Menggabungkan semua hasil ke dalam satu DataFrame
combined_results = pd.DataFrame()

for wilayah in df_results:
    df_region = df_results[wilayah].copy()
    df_region['Wilayah'] = wilayah
    df_region.reset_index(drop=True, inplace=True)
    
    # Mendapatkan daftar kolom
    columns = df_region.columns.tolist()
    columns.remove('Minggu')
    columns.remove('Wilayah')
    
    # Mengekstrak kategori dari nama kolom
    categories = set()
    for col in columns:
        base = col.split(' - ')[0]
        categories.add(base)
    
    # Loop melalui setiap kategori
    for kategori in categories:
        # Memilih kolom untuk kategori ini
        cols_to_select = ['Minggu', 'Wilayah']
        kategori_cols = [col for col in df_region.columns if col.startswith(kategori)]
        cols_to_select.extend(kategori_cols)
        
        df_kategori = df_region[cols_to_select].copy()
        df_kategori['Kategori'] = kategori
        
        # Mengganti nama kolom
        rename_dict = {f'{kategori}': 'Pemakaian Asli'}
        for model in models:
            old_col = f'{kategori} - {model}'
            if old_col in df_kategori.columns:
                rename_dict[old_col] = model
        df_kategori.rename(columns=rename_dict, inplace=True)
        
        # Mengubah DataFrame menjadi format long
        df_melted = df_kategori.melt(id_vars=['Minggu', 'Wilayah', 'Kategori'], value_vars=['Pemakaian Asli'] + models,
                                     var_name='Model', value_name='Nilai')
        
        # Menambahkan hasil ke combined_results
        combined_results = pd.concat([combined_results, df_melted], ignore_index=True)

# Menampilkan DataFrame gabungan
display(combined_results)


In [ ]:
# Menggabungkan metrik kesalahan ke dalam DataFrame
error_metrics_list = []

for key, metrics in error_metrics.items():
    wilayah, kategori = key
    for model, values in metrics.items():
        temp_dict = {'Wilayah': wilayah, 'Kategori': kategori, 'Model': model}
        temp_dict.update(values)
        error_metrics_list.append(temp_dict)

error_metrics_df = pd.DataFrame(error_metrics_list)

# Menampilkan metrik kesalahan
display(error_metrics_df)
